### Speed Up Your Python Program With Concurrency
sumber bacaan: https://realpython.com/python-concurrency/

In [3]:
pip install requests

You should consider upgrading via the 'C:\Users\EVANGS MAILOA\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import requests
import time


def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jyth

#### Threading Version

In [5]:
import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10490 from https://www.jython.orgRead 10490 from https://www.jython.org

Read 10490 from https://www.jython.org
Read 10490 from https://www.jython.org
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 10490 from https://www.jython.org
Read 276 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 276 from

#### Asyncio Version

In [7]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\EVANGS MAILOA\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [10]:
import asyncio
import time
import aiohttp


async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")

RuntimeError: This event loop is already running

#### Multiprocessing Version

In [11]:
import requests
import multiprocessing
import time

session = None


def set_global_session():
    global session
    if not session:
        session = requests.Session()


def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}:Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

C:\Users\EVANGS MAILOA\AppData\Local\Programs\Python\Python39\lib\multiprocessing\__init__.py:16: RuntimeWarning: coroutine 'download_all_sites' was never awaited
  from . import context


In [1]:
import multiprocessing
import time


def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


if __name__ == "__main__":
    numbers = [50 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")